In [1]:
import logging
import sys
sys.path.append("/home/oscar/projects/ai-sales-agent")
from agents.config.llm_models import AVAILABLE_MODELS
logging.basicConfig(level=logging.INFO)

Voy a importar todos los agentes que utilizaré este workflow. Primero una respuestaen general.

Con este puedo clasificar cual es la intención inicial del usuario al escribir un mensaje.


In [2]:
from agents.kavak_classifiers import IntentClassifier

model = "deepseek-v3"
chat_history = []

intent_classifier = IntentClassifier(model=model)

user_input = "Hola"
chat_history.append({"role": "user", "content": user_input})
intent_response = intent_classifier.classify(user_input)
print(f"Intent: {intent_response['_type']}")
assert intent_response["_type"] == "general", f"Expected 'general', but got {intent_response['_type']}"

INFO:agents.baseagents._basebot:Setting system prompt: Clasifica la siguiente solicitud del cliente en una de estas dos categorías:
- "general"
- "catalog"

Responde con unicamente el diccionario que contiene el tipo de solicitud y no incluyas ningún otro texto.

Ejemplos:
Solicitud: ¿Dónde están ubicadas sus sucursales?
Respuesta:
{"_type": "general"}

Solicitud: ¿Estoy buscando un auto pequeñó.
Respuesta:
{"_type": "catalog"}


INFO:agents.baseagents._basebot:Classifying input: Hola
INFO:agents.baseagents._basebot:Sending message: Hola
INFO:agents.baseagents._basebot:Updated chat history with user: Hola
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:agents.baseagents._basebot:Updated chat history with assistant: {"_type": "general"}
INFO:agents.baseagents._basebot:Received response: {"_type": "general"}


Intent: general


In [3]:
from agents.kavak_general import KavakGeneralBot

general_bot = KavakGeneralBot(model=model)
general_response = general_bot.send_message(user_input)
print(f"General Bot Response: {general_response}")
chat_history.append({"role": "assistant", "content": general_response})

INFO:agents.baseagents._basebot:Sending message: Hola
INFO:agents.baseagents._basebot:Updated chat history with user: Hola
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:agents.baseagents._basebot:Updated chat history with assistant: ¡Hola! 😊 ¿En qué puedo ayudarte hoy? Si tienes alguna pregunta sobre Kavak, sus sedes, procesos de compra/venta o financiamiento, con gusto te ayudo.  

Por ejemplo, puedes preguntar:  
- ¿Dónde están las sedes de Kavak en México?  
- ¿Cómo funciona el plan de pagos a meses?  
- ¿Qué beneficios tiene vender mi auto con Kavak?  

Dime qué necesitas saber. 🚗💙
INFO:agents.baseagents._basebot:Received response: ¡Hola! 😊 ¿En qué puedo ayudarte hoy? Si tienes alguna pregunta sobre Kavak, sus sedes, procesos de compra/venta o financiamiento, con gusto te ayudo.  

Por ejemplo, puedes preguntar:  
- ¿Dónde están las sedes de Kavak en México?  
- ¿Cómo funciona el plan de pagos a meses?  
- ¿Qué beneficios tiene vende

General Bot Response: ¡Hola! 😊 ¿En qué puedo ayudarte hoy? Si tienes alguna pregunta sobre Kavak, sus sedes, procesos de compra/venta o financiamiento, con gusto te ayudo.  

Por ejemplo, puedes preguntar:  
- ¿Dónde están las sedes de Kavak en México?  
- ¿Cómo funciona el plan de pagos a meses?  
- ¿Qué beneficios tiene vender mi auto con Kavak?  

Dime qué necesitas saber. 🚗💙


Ahora el usuario va a pedir infomación sobre un producto.

In [4]:
from agents.kavak_classifiers import IntentClassifier

user_input = "Hola estoy buscando un honda cibic 2016"
chat_history.append({"role": "user", "content": user_input})
intent_response = intent_classifier.classify(user_input)
print(f"Intent: {intent_response['_type']}")

assert intent_response["_type"] == "catalog", f"Expected 'catalog', but got {intent_response['_type']}"

INFO:agents.baseagents._basebot:Classifying input: Hola estoy buscando un honda cibic 2016
INFO:agents.baseagents._basebot:Sending message: Hola estoy buscando un honda cibic 2016
INFO:agents.baseagents._basebot:Updated chat history with user: Hola estoy buscando un honda cibic 2016
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:agents.baseagents._basebot:Updated chat history with assistant: {"_type": "catalog"}
INFO:agents.baseagents._basebot:Received response: {"_type": "catalog"}


Intent: catalog


Después for a pasar la pregunta al normalizador, que se encargará de normalizar la pregunta y eliminar palabras innecesarias y dejarla en un formato más limpio para que el clasificador de entidades pueda trabajar mejor.

In [5]:
from agents.vehicle_normalizer import VehicleNormalizer

normalizer = VehicleNormalizer(model=model)
normalized_request = normalizer.normalize(user_input)
normalized_request

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:agents.baseagents._normalizer:Response from LLM: content='```json\n{\n  "marca": "Honda",\n  "modelo": "Civic",\n  "año": 2016,\n  "version": null,\n  "precio": null,\n  "km": null,\n  "bluetooth": null,\n  "car_play": null,\n  "tamaño": null,\n  "carrocería": null\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 457, 'total_tokens': 542, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 448}, 'prompt_cache_hit_tokens': 448, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '63c7316e-9ac7-4bd7-9ec0-177ceecb863c', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--60b650af-9139-4216-8f04-cb5d2fbe40d5-0' usage_metadata={'input_tokens': 457, 'output_tokens': 85, 'total_tokens': 542, 

{'marca': 'Honda',
 'modelo': 'Civic',
 'año': 2016,
 'version': None,
 'precio': None,
 'km': None,
 'bluetooth': None,
 'car_play': None,
 'tamaño': None,
 'carrocería': None}

Posteriormente voy a obtener la respuesta normalziada y la voy a pasar al recomendador. El cual va a elegir el vehiculo más adecuado para la pregunta del usuario.

In [6]:
from agents.vehicle_recommendation import VehicleRecommender
import pandas as pd


data = pd.read_csv("/home/oscar/projects/ai-sales-agent/data/csv/sample_caso_ai_engineer.csv")

recommender = VehicleRecommender(model=model, data=data)
recommendations = recommender.get_recommendations(user_input, normalized_request)
chat_history.append({"role": "assistant", "content": recommendations["readable"]})
recommendations


INFO:agents.baseagents._basebot:Setting system prompt: Eres un asistente de ventas de autos de la empresa Kavak. Tu tarea es ayudar a los clientes a encontrar el auto que buscan. Se te va a dart una de JSON con las especificaciones del auto que busca el cliente. Tu tarea es recomendar de manera precisa y concisa el auto que mejor se ajuste a las especificaciones del cliente. Al final pregunta si desea un plan de financiamiento para ese auto. Tus respuestas se enviarán por whatsapp, asi que no utilices markdown que no es compatible con whatsapp.
INFO:agents.baseagents._basebot:Generating explanation for recommendation: [{'stock_id': 299048, 'km': 64638, 'precio': 192999.0, 'marca': 'Honda', 'modelo': 'Civic', 'año': 2013, 'version': '1.8 EX-L AT 4DRS', 'bluetooth': 'Sí', 'largo': 4555.0, 'ancho': 1755.0, 'altura': 1450.0, 'car_play': nan, 'similarity_score': 97.72727272727272}]
INFO:agents.baseagents._basebot:Context: El usuario preguntó: Hola estoy buscando un honda cibic 2016. 
 Las r

{'raw': [{'stock_id': 299048,
   'km': 64638,
   'precio': 192999.0,
   'marca': 'Honda',
   'modelo': 'Civic',
   'año': 2013,
   'version': '1.8 EX-L AT 4DRS',
   'bluetooth': 'Sí',
   'largo': 4555.0,
   'ancho': 1755.0,
   'altura': 1450.0,
   'car_play': nan,
   'similarity_score': 97.72727272727272}],
 'readable': '¡Hola! Encontramos un Honda Civic que se ajusta a lo que buscas:\n\n*Modelo:* Honda Civic 1.8 EX-L AT 4DRS 2013  \n*Kilometraje:* 64,638 km  \n*Precio:* $192,999  \n*Incluye:* Bluetooth  \n\nAunque es un modelo 2013 (no encontramos exactamente 2016), es muy similar a lo que solicitaste. ¿Te interesa que te cotice un plan de financiamiento para este auto?'}

Se entrega la respuesta al cliente y se espera a su respuesta. Ahora toca clasificar la respuesta del cliente si es que desea un financiamiento o no.

In [7]:
from agents.kavak_classifiers import FinancingBoolClassifier

user_response = "Si, me gustaría financiarlo"
chat_history.append({"role": "user", "content": user_response})
financing_classifier = FinancingBoolClassifier(model=model)
response = financing_classifier.classify(user_response)
print(f"Financing: {response['response']}")

assert response["response"] == True, f"Expected True, but got {response['response']}"

INFO:agents.baseagents._basebot:Setting system prompt: Eres un asistente que clasifica las respuestas de los clientes. El cliente te va a responder si desea un plan de financiamiento o no. Tu tarea es clasificar la respuesta en dos categorias: true  o false. Si el cliente dice que si, entonces devuelve true , si el cliente dice que no, entonces devuelve false. Responde unicamente con el diccionario JSON que contiene la respuesta. No incluyas ningun otro texto.
Ejemplo:

El cliente dice: "si, me interesa un plan de financiamiento"
Respuesta: 
{"response": true}
INFO:agents.baseagents._basebot:Classifying input: Si, me gustaría financiarlo
INFO:agents.baseagents._basebot:Sending message: Si, me gustaría financiarlo
INFO:agents.baseagents._basebot:Updated chat history with user: Si, me gustaría financiarlo
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:agents.baseagents._basebot:Updated chat history with assistant: ```json
{"response": true}

Financing: True


El agente pregunta que a cuantos años y con cuanto de enganche. El cliente response y se calcula el financiamiento.

In [8]:
from agents.kavak_classifiers import FinancingTermClassifier
from agents.utils.financing_plan import calc_financing

term_question = "¿A cuantos años quieres financiarlo y con cuanto de enganche?"
chat_history.append({"role": "user", "content": term_question})

model = "deepseek-v3"       
financing_term_classifier = FinancingTermClassifier(model=model)
term = "3 años y 20% de enganche"
chat_history.append({"role": "user", "content": term})
financing_term_response = financing_term_classifier.classify(term)
print(f"Financing Term: {financing_term_response}")

INFO:agents.baseagents._basebot:Setting system prompt: CLasifica la solicitud del client en 3, 4, 5, o 6 años y cuanto quiere dar de enganche. si nmo pudes extraer el dato, entonces devuelve null. Responde unicamente con el diccionario JSON que contiene la respuesta. No incluyas ningun otro texto.
Considera que el cliente puede usar numeros, letras, o una combinacion de ambos.

Ejemplo:
El cliente dice: "a 3 años con un enganche de 10000"
Respuesta: 
{"term": 3, "down_payment": 10000}

El cliente dice: "a 3 años con un enganche de 10%"
Respuesta: 
{"term": 3, "down_payment": precio * 0.1}

El cliente dice: "me gustaría financiarlo a 4 meses con un enganche de 10000" # meses no es valido
Respuesta:
{"term": null, "down_payment": 10000}

El cliente dice: "me gustaría financiarlo a 10 años con un enganche de 10000"  # No es valido mas de 6 años
Respuesta:
{"term": null, "down_payment": 10000}

El cliente dice: "a 2"
Respuesta:
{"term": null, "down_payment": null}

El cliente dice: "me gus

Financing Term: {'term': 3, 'down_payment': 'precio*0.2', 'input': '3 años y 20% de enganche'}


Finalmente el agene toma el calculo y lo explica en un formato más amigable para el cliente.

In [9]:
from agents.financing_explainer import FinancingExplainer


financing_plan = calc_financing(
    total_price=1000000,
    down_payment=financing_term_response["down_payment"],
    term=financing_term_response["term"],
    )
financing_plan

explainer = FinancingExplainer(model=model)
financing_explanation = explainer.send_message(str(financing_plan))
financing_explanation

chat_history.append({"role": "assistant", "content": financing_explanation})

INFO:agents.baseagents._basebot:Sending message: {'precio': 1000000, 'enganche': 200000.0, 'años': 3, 'mensualidad': 25813.75, 'total_pagado': 929294.99, 'intereses_totales': 129294.99, 'monto_financiar': 800000.0}
INFO:agents.baseagents._basebot:Updated chat history with user: {'precio': 1000000, 'enganche': 200000.0, 'años': 3, 'mensualidad': 25813.75, 'total_pagado': 929294.99, 'intereses_totales': 129294.99, 'monto_financiar': 800000.0}
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:agents.baseagents._basebot:Updated chat history with assistant: ¡Claro! Aquí tienes el resumen de tu financiamiento:  

🔹 **Precio del auto:** $1,000,000 MXN  
🔹 **Enganche (20%):** $200,000 MXN  
🔹 **Monto a financiar:** $800,000 MXN  
🔹 **Plazo:** 3 años (36 meses)  
🔹 **Pago mensual:** $25,813.75 MXN  
🔹 **Total a pagar (incluyendo intereses):** $929,294.99 MXN  
🔹 **Intereses totales:** $129,294.99 MXN  

En resumen:  
- Pagarás $25,813.75 al mes duran

In [10]:
from pprint import pprint  

print("Chat History")
pprint(chat_history)

Chat History
[{'content': 'Hola', 'role': 'user'},
 {'content': '¡Hola! 😊 ¿En qué puedo ayudarte hoy? Si tienes alguna pregunta '
             'sobre Kavak, sus sedes, procesos de compra/venta o '
             'financiamiento, con gusto te ayudo.  \n'
             '\n'
             'Por ejemplo, puedes preguntar:  \n'
             '- ¿Dónde están las sedes de Kavak en México?  \n'
             '- ¿Cómo funciona el plan de pagos a meses?  \n'
             '- ¿Qué beneficios tiene vender mi auto con Kavak?  \n'
             '\n'
             'Dime qué necesitas saber. 🚗💙',
  'role': 'assistant'},
 {'content': 'Hola estoy buscando un honda cibic 2016', 'role': 'user'},
 {'content': '¡Hola! Encontramos un Honda Civic que se ajusta a lo que '
             'buscas:\n'
             '\n'
             '*Modelo:* Honda Civic 1.8 EX-L AT 4DRS 2013  \n'
             '*Kilometraje:* 64,638 km  \n'
             '*Precio:* $192,999  \n'
             '*Incluye:* Bluetooth  \n'
             '\n'
      